In [52]:
from numpy.linalg import inv, svd
import math

def fit(X, y, lambda_):
    theta = {}

    U, s, V = svd(X.T)
    D = np.diag(s)
    Z = U.dot(D)
    Sigma_z_hat = np.cov(Z, rowvar = False, ddof = 0)
    Sigma_z_tilde = lambda_ * np.diag(Sigma_z_hat) + (1 - lambda_) * Sigma_z_hat
    
    for c in np.unique(y):
        prior = np.mean(y == c)
        mu = V.T.dot(np.mean(X[y == c], axis = 0))
        Beta = V.dot(inv(Sigma_z_tilde)).dot(mu)
        gamma = mu.T.dot(Beta) + math.log(prior)
        theta[c] = (Beta, gamma)
        
fit(X, y, 0.3)

ValueError: shapes (210,210) and (2,) not aligned: 210 (dim 1) != 2 (dim 0)

In [50]:
print np.cov(X, rowvar = False, ddof = 0)
mu = np.mean(X, axis = 0)
mu.shape = (2, 1)
print X.T.dot(X) / (len(X) * 1.0) - mu.dot(mu.T)
M = np.zeros((2, 2))
for x in X:
    x.shape = (2, 1)
    M += (x - mu).dot((x - mu).T)
M /= len(X)
M

[[   19.78331066   104.01931973]
 [  104.01931973  1063.86340136]]
[[   19.78331066   104.01931973]
 [  104.01931973  1063.86340136]]


array([[   19.78331066,   104.01931973],
       [  104.01931973,  1063.86340136]])

In [49]:
print X.T.dot(X) / (len(X) * 1.0)
M = np.zeros((2, 2))
for x in X:
    x.shape = (2, 1)
    M += (x).dot((x).T) * 1.0
M / len(X)

[[  4557.40952381   9915.76190476]
 [  9915.76190476  22279.86666667]]


array([[  4557.40952381,   9915.76190476],
       [  9915.76190476,  22279.86666667]])

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
# from sklearn import qda, lda
import math
from numpy.linalg import inv
import lda
import qda
import lr
import htwt

In [3]:
X, y = htwt.get()

In [3]:
model = lr.fit(X, y)
np.mean(lr.predict(model, X) == y)

0.87142857142857144

In [4]:
model = qda.fit(X, y)
np.mean(qda.predict(model, X) == y)

0.88095238095238093

In [5]:
X = (X - np.mean(X, axis = 0)) / np.std(X, axis = 0)
model= lda.fit(X, y)
np.mean(lda.predict(model, X) == y)

0.87142857142857144